In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%config Completer.use_jedi = False
import warnings 
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
trainp = "/kaggle/input/titanic/train.csv"
testp="/kaggle/input/titanic/test.csv"

# Reading data

In [ ]:
dftr = pd.read_csv(trainp)
dftest = pd.read_csv(testp)
dftr.head()

## Understanding data

In [ ]:
dftr.info()

In [ ]:
dftr.describe()

In [ ]:
dftr.columns

# dropping columns

In [ ]:
drop_col = ["PassengerId","Name","Ticket",'Cabin']
dftr1 = dftr.drop(drop_col,axis=1)
dftr1.columns

# Handling missing data

In [ ]:
dftr1.isnull().sum()

In [ ]:
dftr1.Embarked.value_counts()

In [ ]:
dftr1["Embarked"] = dftr1["Embarked"].fillna(dftr1.Embarked.mode()[0])
dftr1["Age"] = dftr1["Age"].fillna(dftr1.Age.mean())[0]
dftr1.Embarked.value_counts()

In [ ]:
cc = ["Embarked","Sex"]
dftr_dumm = pd.get_dummies(dftr1[cc])
dftr_dumm.head()

In [ ]:
dftr2 = dftr1.drop(cc,axis=1)
dftr2 = pd.concat([dftr2,dftr_dumm],axis=1)
dftr2.info()

# features and target

In [ ]:
X = dftr2.iloc[:,1:] 
y = dftr2.iloc[:,0]


## using Tree classifier without scaling

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
sn = []
score1 = []
for i in range(1,101):
    X_train, X_valid, ytrain,yvalid = train_test_split(X,y,stratify=y,test_size=.25)
    sn.append(i)
    model.fit(X_train,ytrain)
    score1.append(float('{:.2f}'.format(model.score(X_valid,yvalid)*100)))
#     print("%.2f" % (model.score(X_valid,yvalid)*100))

In [ ]:
result = pd.DataFrame({"sn":sn,"score":score1})
r1=float('%.2f'%result.score.mean())
print("Tree classifier score %s "%(r1))
result.to_csv("Tree_classifier_score.csv")

## using MinMaxScaler with Tree classifier

In [ ]:
minmax = MinMaxScaler()
minmax.fit(X)
Xt_mmscaled = minmax.transform(X)

In [ ]:
score2 = []
for i in range(1,101):
    Xsmin_train, Xsmin_valid, ytrain,yvalid = train_test_split(Xt_mmscaled,y,stratify=y,test_size=.25)
    model.fit(Xsmin_train,ytrain)
    score2.append(float('{:.2f}'.format(model.score(Xsmin_valid,yvalid)*100)))
#     print("%.2f" % (model.score(Xsmin_valid,yvalid)*100))

In [ ]:
result = pd.DataFrame({"sn":sn,"score":score2})
r2=float('%.2f'%result.score.mean())
print("Tree classifier with MinMax score %s "%(r2))
result.to_csv("MinMax_to_Tree_classifier_score.csv")

## using StandardScaler with Tree classifier

In [ ]:
standard = StandardScaler()
standard.fit(X)
Xt_sscaled = standard.transform(X)

In [ ]:
score3 = []
for i in range(1,101):
    Xsst_train, Xsst_valid, ytrain,yvalid = train_test_split(Xt_sscaled,y,stratify=y,test_size=.25)
    model.fit(Xsst_train,ytrain)
    mscore = model.score(Xsst_valid,yvalid)*100
    score3.append(float('{:.2f}'.format(mscore)))
#     print("%.2f" % (mscore))

In [ ]:
result = pd.DataFrame({"sn":sn,"score":score3})
r3=float('%.2f'%result.score.mean())
print("Tree classifier with StandardScaler score %s "%(r3))
result.to_csv("StandardScaler_Tree_classifier_score.csv")

### plotting all three

In [ ]:
plt.plot(sn,score1,label=["epoc","sc"])
plt.plot(sn,score2,label=["epoc","sc"])
plt.plot(sn,score3,label=["epoc","sc"])
plt.legend(["Tree","Minmax","Standard"])

In [ ]:

plt.plot(["Tree","Minmax","Standard"],[r1,r2,r3])

plt.grid()
plt.xticks(rotation=45)

|# Lab on 13 May |
=======================

### MinMax scaled data has been used with following below:-

### Random Forest , param n_estimators specifies the number of Tree estimators to be used

In [ ]:
rf_model = RandomForestClassifier(n_estimators=30)

In [ ]:
score_rfo = []    
for i in range(1,101):
    Xsmin_train, Xsmin_valid, ytrain,yvalid = train_test_split(Xt_mmscaled,y,stratify=y,test_size=.25)
    rf_model.fit(Xsmin_train,ytrain)
    mscore = rf_model.score(Xsmin_valid,yvalid)*100
    score_rfo.append(float('{:.2f}'.format(mscore)))
    #print("%.2f" % (mscore))

In [ ]:
result_rfo = pd.DataFrame({"sn":sn,"score":score_rfo})
result_rfo=float('%.2f'%result_rfo.score.mean())
print("RandomForest classifier with MinMax score %s "%(result_rfo))
result.to_csv("RandomForest_classifier_score.csv")

### KNN

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=4)

In [ ]:
score_knn = []    
for i in range(1,101):
    Xsmin_train, Xsmin_valid, ytrain,yvalid = train_test_split(Xt_mmscaled,y,stratify=y,test_size=.25)
    knn_model.fit(Xsmin_train,ytrain)
    knn_score = knn_model.score(Xsmin_valid,yvalid)*100
    score_knn.append(float('{:.2f}'.format(knn_score)))
    #print("%.2f" % (mscore))

In [ ]:
result_knn = pd.DataFrame({"sn":sn,"score":score_knn})
result_knn=float('%.2f'%result_knn.score.mean())
print("KNN classifier with MinMax score %s "%(result_knn))
result.to_csv("KNN_classifier_score.csv")

### SVM

In [ ]:
svc_model = SVC(kernel='linear')

In [ ]:
score_svc = []    
for i in range(1,101):
    Xsmin_train, Xsmin_valid, ytrain,yvalid = train_test_split(Xt_mmscaled,y,stratify=y,test_size=.25)
    svc_model.fit(Xsmin_train,ytrain)
    svm_score = svc_model.score(Xsmin_valid,yvalid)*100
    score_svc.append(float('{:.2f}'.format(svm_score)))
    #print("%.2f" % (mscore))

In [ ]:
result_svc = pd.DataFrame({"sn":sn,"score":score_svc})
result_svc=float('%.2f'%result_svc.score.mean())
print("SVC classifier with MinMax score %s "%(result_svc))
result.to_csv("SVC_classifier_score.csv")

### Naive Bayes

In [ ]:
nav_model = GaussianNB()

In [ ]:
score_naiv = []    
for i in range(1,101):
    Xsmin_train, Xsmin_valid, ytrain,yvalid = train_test_split(Xt_mmscaled,y,stratify=y,test_size=.25)
    nav_model.fit(Xsmin_train,ytrain)
    naiv_score = nav_model.score(Xsmin_valid,yvalid)*100
    score_naiv.append(float('{:.2f}'.format(naiv_score)))
    #print("%.2f" % (mscore))

In [ ]:
result_naiv = pd.DataFrame({"sn":sn,"score":score_naiv})
result_naiv=float('%.2f'%result_naiv.score.mean())
print("NaiveBayes classifier with MinMax score %s "%(result_naiv))
result.to_csv("NaivBayes_classifier_score.csv")

## Plotting Decision Tree(score2) , RandForest(score_rfo), KNN(score_knn), SVC(score_svc), NaiveBayes(score_naiv)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(sn,score2,label=["epoc","sc"])
plt.plot(sn,score_rfo,label=["epoc","sc"])
plt.plot(sn,score_knn,label=["epoc","sc"])
plt.plot(sn,score_svc,label=["epoc","sc"])
plt.plot(sn,score_naiv,label=["epoc","sc"])
plt.legend(["Tree","Forest","KNN",'SVC',"Naiv"])